In [1]:
# 허깅페이스(huggingface) 의 transformers 라이브러리 
!pip install transformers

In [2]:
!pip install sentencepiece

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import *

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [4]:
# https://github.com/e9t/nsmc/
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [5]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 버트에 필요한 입력값 요약
- input_ids : 문장을 토크나이즈해서 인덱스 값으로 변환한다. 일반적으로 버트에서는 단어를 서브 워드의 단위로 변환시키는 워드 피스 토크나이저를 활용한다 
- attention_mask : 어텐션 마스크는 패딩된 부분에 대해 학습에 영향을 받지 않기 위해 처리해주는 입력값이다. 버트 토크나이저에서 1은 어텐션에 영향을 받는 토큰을 0은 영향을 받지 않는 토큰을 나타낸다 
- token_type_ids : 두개의 시퀀스 입력을 활용할 때 0가 1로 문장의 토큰 값을 분리한다 

### 버트의 각 스페셜 토큰의 역할
- [UNK] : 모르는 단어에 대한 토튼
- [MASK] : 마스크 토큰, 사전 학습에서 활용
- [PAD] : 최대 길이를 맞추는 용도
- [SEP] : 문장의 종결을 알림
- [CLS] : 문장의 시작을 알림

In [6]:
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # add [CLS] and [SEP]
        max_length = MAX_LEN,
        pad_to_max_length = True,
        return_attention_mask = True
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

# eocoded_plus 순서 
# 1. 문장을 토크나이징 한다 
# 2. add_special_tokens 를 True로 지정하면 토큰의 시작점에 [CLS] 토큰, 마지막에는 [SEP] 를 붙인다 
# 3. 각 토큰을 인덱스로 변환한다 
# 4. max_length에 MAX_LEN 최대 길이에 따라 문장의 길이를 맞추는 작업을 진행하고, 
#    pad_to_max_length 기능을 통해 MAX_LEN 의 길이에 미치지 못하는 문장에 패딩을 적용 
# 5. return_attention_mask 기능을 통해 어텐션 마스크를 생성
# 토큰 타입은 문장이 1개일 경우 0으로, 문장의 2개 일 경우 0과 1로 구분해서 생성 

# 버트를 활용한 한국어 텍스트 분류

#### 네이버 영화 리뷰 데이터 전처리 

In [7]:
# special tokens
print(tokenizer.all_special_tokens, "\n", tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] 
 [100, 102, 0, 101, 103]


In [8]:
# test tokenizers
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다")
eng_encode = tokenizer.encode("hello world")
kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

In [9]:
print(kor_encode)

[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]


In [10]:
print(eng_encode)

[101, 61694, 10133, 11356, 102]


In [11]:
print(kor_decode)

[CLS] 안녕하세요, 반갑습니다 [SEP]


In [12]:
print(eng_decode)

[CLS] hello world [SEP]


In [13]:
MAX_LEN=50

In [14]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

for train_sent, train_label in zip(train_data["document"], train_data["label"]):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)
        
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_movie_input_ids = np.array(input_ids, dtype=int)
train_movie_attention_masks = np.array(attention_masks, dtype=int)
train_movie_type_ids = np.array(token_type_ids, dtype= int)
train_movie_inputs = (train_movie_input_ids,train_movie_attention_masks, train_movie_type_ids )

# 정답 토크나이징 리스트 
train_data_labels = np.asarray(train_data_labels, dtype=np.int32)

print("sent : {}, # labels : {}".format(len(train_movie_input_ids), len(train_data_labels)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
sent : 149995, # labels : 149995


In [15]:
# Max_length = 39 
input_id = train_movie_input_ids[1]
attention_mask = train_movie_attention_masks[1]
token_type_ids = train_movie_type_ids[1]

print(input_id)
print(attention_mask)
print(token_type_ids)
print(tokenizer.decode(input_id))

[   101    100   9928  58823  30005  11664   9757 118823  30858  18227
 119219   9580  41605  25486  12310  20626  23466   8843 118986  12508
   9523  17196  16439    102      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[CLS] [UNK] 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [ ]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=2)


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [ ]:
# 학습 준비하기
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=2)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

# 학습과 eval 시작
history = cls_model.fit(train_inputs,
                        train_labels,
                        epochs=10, 
                        batch_size=32,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])